<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/VQA_BioGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -q install fairseq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 94.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.4/240.4 KB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 KB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [1]:
!pip -q install moses

  Preparing metadata (setup.py) ... done


In [3]:
!pip -q install fastBPE

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp38-cp38-linux_x86_64.whl size=759287 sha256=233480930393f393adb81ccdb917bdfbcb1c6510f9daaf7246092455deb260e3
  Stored in directory: /root/.cache/pip/wheels/64/a8/c6/07fb6443539116f05252337e751bbccd2bd2dd6c585ba94ca8
Successfully built fastBPE


In [4]:
!pip -q install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [5]:
!git clone https://github.com/microsoft/BioGPT.git
%cd BioGPT

Cloning into 'BioGPT'...
remote: Enumerating objects: 299, done.
remote: Counting objects: 100% (299/299), done.
remote: Compressing objects: 100% (179/179), done.
remote: Total 299 (delta 150), reused 228 (delta 109), pack-reused 0
Receiving objects: 100% (299/299), 31.43 MiB | 35.41 MiB/s, done.
Resolving deltas: 100% (150/150), done.
/content/BioGPT


In [6]:
!mkdir checkpoints
%cd checkpoints
!wget https://msramllasc.blob.core.windows.net/modelrelease/BioGPT/checkpoints/Pre-trained-BioGPT.tgz
!tar -zxvf Pre-trained-BioGPT.tgz
%cd ..

/content/BioGPT/checkpoints
--2023-02-03 11:31:29--  https://msramllasc.blob.core.windows.net/modelrelease/BioGPT/checkpoints/Pre-trained-BioGPT.tgz
Resolving msramllasc.blob.core.windows.net (msramllasc.blob.core.windows.net)... 20.209.34.164
Connecting to msramllasc.blob.core.windows.net (msramllasc.blob.core.windows.net)|20.209.34.164|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3414384105 (3.2G) [application/octet-stream]
Saving to: ‘Pre-trained-BioGPT.tgz’

Pre-trained-BioGPT. 100%[===================>]   3.18G  21.6MB/s    in 2m 15s  

2023-02-03 11:33:43 (24.2 MB/s) - ‘Pre-trained-BioGPT.tgz’ saved [3414384105/3414384105]

Pre-trained-BioGPT/
Pre-trained-BioGPT/checkpoint.pt
/content/BioGPT


In [11]:
import torch
from fairseq.models.transformer_lm import TransformerLanguageModel
m = TransformerLanguageModel.from_pretrained(
        "checkpoints/Pre-trained-BioGPT", 
        "checkpoint.pt", 
        "data",
        tokenizer='moses', 
        bpe='fastbpe', 
        bpe_codes="data/bpecodes",
        min_len=100,
        max_len_b=1024)
# m.cuda()
src_tokens = m.encode("COVID-19 is a disease")
generate = m.generate([src_tokens], beam=5)[0]
output = m.decode(generate[0]["tokens"])
print(output)

COVID-19 is a disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2), which was first identified in Wuhan, China, in December 2019 and has since spread to more than 200 countries and territories, including the United States, Canada, the United Kingdom, the United States of America, the United States of America, Australia, New Zealand, Canada, and the United States of America, as well as the rest of the world, including Europe, Asia, Africa, and South America.


In [30]:
src_tokens = m.encode("COVID-19 is not a")
generate = m.generate([src_tokens], beam=4)[0]
output = m.decode(generate[0]["tokens"])
print(len(generate))
print(output)
print(generate[0]['positional_scores'], generate[0]['positional_scores'].shape)

4
COVID-19 is not a single disease, but rather a group of diseases that are caused by a novel coronavirus, SARS-CoV-2, which was first identified in December 2019 in Wuhan, Hubei Province, China, and has since spread rapidly to other parts of the world, including the United States, Canada, and the United Kingdom, as well as other countries in Asia, Europe, Africa, and the Middle East, as well as to the United States, Canada, and the United States.
tensor([-1.0468e+01, -1.4261e-02, -3.3998e-03, -2.5903e+00, -4.9233e+00,
        -1.4998e+00, -2.1849e+00, -4.5880e-01, -1.1768e+00, -2.0609e-01,
        -1.0960e+00, -3.6957e-01, -1.9237e+00, -1.2863e-02, -1.3079e+00,
        -1.7819e+00, -1.6281e+00, -1.0102e+00, -1.0113e-02, -7.9506e-01,
        -6.9929e-01, -2.6900e-01, -1.1905e+00, -3.8622e-01, -1.1929e-02,
        -3.9898e-02, -6.1951e-03, -1.9455e-03, -1.4751e+00, -1.1080e+00,
        -1.8896e+00, -4.0522e-01, -5.4012e-01, -5.1334e-02, -1.3673e+00,
        -1.4217e-02, -4.1369e-01, -4.

In [28]:
src_tokens = m.encode("COVID-19 is a disease originated from")
generate = m.generate([src_tokens], beam=5)[0]
output = m.decode(generate[0]["tokens"])
print(len(generate))
print(output)
print(generate[0]['positional_scores'], generate[0]['positional_scores'].shape)

5
COVID-19 is a disease originated from severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2), which was first identified in Wuhan, China, in December 2019 and has since spread to more than 200 countries and territories, including the United States, Canada, Australia, New Zealand, the United Kingdom, the United States of America, the United States of America, the United Kingdom, France, Germany, Italy, Spain, the United Kingdom, and the United States of America, including the United States of America.
tensor([-1.0468e+01, -1.4261e-02, -3.3998e-03, -2.5903e+00, -6.5561e-01,
        -3.6940e+00, -6.7434e+00, -1.3912e+00, -3.2693e+00, -5.2650e-02,
        -4.1676e-03, -1.1541e-02, -1.1666e-01, -1.8737e-01, -2.6432e-01,
        -8.8348e-03, -2.4990e-02, -2.6752e-02, -7.5588e-03, -4.6539e-04,
        -1.7010e-02, -1.1486e+00, -7.0353e-01, -2.0638e+00, -4.8047e-01,
        -8.1303e-01, -6.4625e-02, -6.8348e-01, -1.5640e-04, -2.1694e-01,
        -1.7050e-01, -5.9023e-01, -2.7578e-01, -